In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.INFO)
from common import find_mxnet, data, fit
from common.util import download_file
import mxnet as mx
import numpy as np
from mxnet.symbol import *

In [2]:
def download_cifar100():
    data_dir="/efs/data/cifar-100-mxnet"
    #data_dir="/data/cifar-100-mxnet"
    fnames = (os.path.join(data_dir, "train.rec"),
              os.path.join(data_dir, "test.rec"))
    return fnames

In [3]:
(train_fname, val_fname) = download_cifar100()

# parse args
parser = argparse.ArgumentParser(description="train cifar100",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
fit.add_fit_args(parser)
data.add_data_args(parser)
data.add_data_aug_args(parser)
data.set_data_aug_level(parser, 2)

num_unsup_examples = 50000
subset_factor = 5

parser.set_defaults(
    # data
    data_train     = train_fname,
    data_val       = val_fname,
    num_classes    = 100,
    image_shape    = '3,28,28',
 #   model_prefix   = '/efs/checkpoints/inceptionStandard',
    pad_size       = 4,
    # train
    batch_size     = 128,
    num_epochs     = 150,
    lr_step_epochs = '80,120',
    gpus           = '0,1,2,3',
    disp_batches   = 10,
    num_examples   = num_unsup_examples / subset_factor, 
    wd             = 1e-4,
    lr             = .05,
    lr_factor      = .33,
    optimizer      = 'sgd'
)

args = parser.parse_args("")
unsup_multiplier = 0
labeled_per_class = 500 / subset_factor
sample_seed = 47
val_interval = 1
num_sup_examples = labeled_per_class * args.num_classes
sup_batch_size = args.batch_size

In [4]:
from common.multi_iterator import Multi_iterator
from common.data import get_partial_rec_iter

(train_sup, val) = get_partial_rec_iter(args, get_val=True, devide_by=5, shuffle=True)
(train_unsup, _) = get_partial_rec_iter(args, get_val=False, devide_by=1, shuffle=True)
    
train = Multi_iterator(train_sup, train_unsup, unsup_multiplier, num_unsup_examples, num_sup_examples)

In [5]:
train.next().data

[<NDArray 128x3x28x28 @cpu(0)>]

In [6]:
from symbols import inception_cifar as inception
from common.lba import compute_semisup_loss, logit_loss

t_nb = sup_batch_size * unsup_multiplier

def build():
    dataSup = mx.symbol.Variable(name="dataSup")
    labelSup = mx.symbol.Variable(name='labelSup')
    overall_loss = []
    
    if unsup_multiplier >= 1:
        dataUnsup = []
        for i in range(unsup_multiplier):
            dataUnsup.append(Variable(name="dataUnsup"+str(i)))

        # concat data, feed both through the network
        # then split it up again
        data = concat(dataSup, *dataUnsup, dim=0)

        embeddings = inception.build_embeddings(data, nembeddings=256)
        splitted = split(embeddings, num_outputs=(unsup_multiplier+1), axis=0, name='split')

        supEmbeddings = splitted[0]
        
        unsupEmbeddings = []
        for i in range(unsup_multiplier):
            unsupEmbeddings.append(splitted[1+i])
    
        unsupEmbeddings = concat(*unsupEmbeddings, dim=0)
            
        (walker_loss, visit_loss) = compute_semisup_loss(supEmbeddings, unsupEmbeddings, labelSup, t_nb, 
                                                     walker_weight=1.0, visit_weight=1.0)
        overall_loss = [walker_loss, visit_loss]
        
    else:
        supEmbeddings = inception.build_embeddings(dataSup, nembeddings=256)
        
    overall_loss = [logit_loss(supEmbeddings, labelSup, 100)] + overall_loss
    
    return Group(overall_loss)

In [7]:
class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):

        #for i in range(len(preds)):
        for i in range(1):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[0].asnumpy().astype('int32')

            #mx.metric.check_label_shapes(label, pred_label)
            
            #print((pred_label.flat == label.flat).sum())
            #print(len(pred_label.flat))

            
            self.sum_metric[i] += (pred_label.flat == label.flat).sum()
            self.num_inst[i] += len(pred_label.flat)

In [ ]:
# evaluation metrices
eval_metrics = Multi_Accuracy(num= 3 if unsup_multiplier >= 1 else 1)
                    
def fit_model(args, network, data, **kwargs):
    """
    train a model
    args : argparse returns
    network : the symbol definition of the nerual network
    data_loader : function that returns the train and val data iterators
    """
    # kvstore
    kv = mx.kvstore.create(args.kv_store)

    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    logging.basicConfig(level=logging.DEBUG, format=head)
    logging.info('start with arguments %s', args)
    batch_size = sup_batch_size * (unsup_multiplier + 1)

    # data iterators
    (train, val) = data
    if args.test_io:
        tic = time.time()
        for i, batch in enumerate(train):
            for j in batch.data:
                j.wait_to_read()
            if (i+1) % args.disp_batches == 0:
                logging.info('Batch [%d]\tSpeed: %.2f samples/sec' % (
                    i, args.disp_batches*batch_size/(time.time()-tic)))
                tic = time.time()
        return

    # load model
    if 'arg_params' in kwargs and 'aux_params' in kwargs:
        arg_params = kwargs['arg_params']
        aux_params = kwargs['aux_params']
    else:
        sym, arg_params, aux_params = fit._load_model(args, kv.rank)
        if sym is not None:
            assert sym.tojson() == network.tojson()

    # save model
    checkpoint = fit._save_model(args, kv.rank)

    # devices for training
    devs = mx.cpu() if args.gpus is None or args.gpus is '' else [
        mx.gpu(int(i)) for i in args.gpus.split(',')]

    # learning rate
    lr, lr_scheduler = fit._get_lr_scheduler(args, kv)

    data_names = ['dataSup'] + ['dataUnsup'+str(i) for i in range(unsup_multiplier)]
        
    # create model
    model = mx.mod.Module(
        context       = devs,
        symbol        = network,
        label_names   = ['labelSup'],
        data_names   = data_names
    )
    
    def validate_model(epoch, *args):
        if epoch % val_interval != 0: 
            return
        res = model.score(val, eval_metrics)
        #TODO: pull this into default
        for name, value in res:
            print('Epoch[%d] Validation-%s=%f' % (epoch, name, value))

    
    #print(model.label_names)
    lr_scheduler  = lr_scheduler
    optimizer_params = {
            'learning_rate': lr,
            'momentum' : args.mom,
            'wd' : args.wd,
            #'rescale_grad': 1,
            'lr_scheduler': lr_scheduler}

    #monitor = mx.mon.Monitor(interval=1000, pattern='.*aba_backward.*') 
    monitor = mx.mon.Monitor(interval=1000, pattern='.*') 
    
    # callbacks that run after each batch
    batch_end_callbacks = [mx.callback.Speedometer(batch_size, args.disp_batches)]
    
    epoch_end_callbacks = validate_model

    # run
    model.fit(train,
        begin_epoch        = args.load_epoch if args.load_epoch else 0,
        num_epoch          = args.num_epochs,
        #eval_data          = val,
        eval_metric        = eval_metrics,
        kvstore            = kv,
        optimizer          = args.optimizer,
        optimizer_params   = optimizer_params,
        #initializer        = initializer,
        arg_params         = arg_params,
        aux_params         = aux_params,
        batch_end_callback = batch_end_callbacks,
        epoch_end_callback = epoch_end_callbacks,
        allow_missing      = True
        #monitor            = monitor
             )
    return model

In [ ]:
train_sup.reset()
train_unsup.reset()

net = build()

model = fit_model(args, net, (train, val))

INFO:root:start with arguments Namespace(batch_size=128, benchmark=0, data_nthreads=4, data_train='/efs/data/cifar-100-mxnet/train.rec', data_val='/efs/data/cifar-100-mxnet/test.rec', disp_batches=10, dtype='float32', gpus='0,1,2,3', image_shape='3,28,28', kv_store='device', load_epoch=None, lr=0.05, lr_factor=0.33, lr_step_epochs='80,120', max_random_aspect_ratio=0, max_random_h=36, max_random_l=50, max_random_rotate_angle=0, max_random_s=50, max_random_scale=1, max_random_shear_ratio=0, min_random_scale=1, model_prefix=None, mom=0.9, monitor=0, network=None, num_classes=100, num_epochs=150, num_examples=10000.0, num_layers=None, optimizer='sgd', pad_size=4, random_crop=1, random_mirror=1, rgb_mean='123.68,116.779,103.939', test_io=0, top_k=0, wd=0.0001)


[6250.0, 9375.0]


INFO:root:Epoch[0] Batch [10]	Speed: 466.34 samples/sec	Train-multi-accuracy_0=0.019886
INFO:root:Epoch[0] Batch [20]	Speed: 435.24 samples/sec	Train-multi-accuracy_0=0.020313
INFO:root:Epoch[0] Batch [30]	Speed: 437.07 samples/sec	Train-multi-accuracy_0=0.022656
INFO:root:Epoch[0] Batch [40]	Speed: 437.01 samples/sec	Train-multi-accuracy_0=0.017969
INFO:root:Epoch[0] Batch [50]	Speed: 436.31 samples/sec	Train-multi-accuracy_0=0.028906
INFO:root:Epoch[0] Batch [60]	Speed: 435.60 samples/sec	Train-multi-accuracy_0=0.024219
INFO:root:Epoch[0] Batch [70]	Speed: 435.95 samples/sec	Train-multi-accuracy_0=0.028906
INFO:root:Epoch[0] Train-multi-accuracy_0=0.030273
INFO:root:Epoch[0] Time cost=25.073


Epoch[0] Validation-multi-accuracy_0=0.030854


INFO:root:Epoch[1] Batch [10]	Speed: 435.94 samples/sec	Train-multi-accuracy_0=0.031250
INFO:root:Epoch[1] Batch [20]	Speed: 435.29 samples/sec	Train-multi-accuracy_0=0.025000
INFO:root:Epoch[1] Batch [30]	Speed: 435.51 samples/sec	Train-multi-accuracy_0=0.027344
INFO:root:Epoch[1] Batch [40]	Speed: 435.80 samples/sec	Train-multi-accuracy_0=0.028125
INFO:root:Epoch[1] Batch [50]	Speed: 435.54 samples/sec	Train-multi-accuracy_0=0.039062
INFO:root:Epoch[1] Batch [60]	Speed: 433.81 samples/sec	Train-multi-accuracy_0=0.034375
INFO:root:Epoch[1] Batch [70]	Speed: 435.10 samples/sec	Train-multi-accuracy_0=0.046875
INFO:root:Epoch[1] Train-multi-accuracy_0=0.046875
INFO:root:Epoch[1] Time cost=22.756


...

Epoch[147] Validation-multi-accuracy_0=0.512320


INFO:root:Epoch[148] Batch [10]	Speed: 432.50 samples/sec	Train-multi-accuracy_0=0.980824
INFO:root:Epoch[148] Batch [20]	Speed: 434.27 samples/sec	Train-multi-accuracy_0=0.976562
INFO:root:Epoch[148] Batch [30]	Speed: 434.54 samples/sec	Train-multi-accuracy_0=0.971875
INFO:root:Epoch[148] Batch [40]	Speed: 434.83 samples/sec	Train-multi-accuracy_0=0.977344
INFO:root:Epoch[148] Batch [50]	Speed: 435.32 samples/sec	Train-multi-accuracy_0=0.976562
INFO:root:Epoch[148] Batch [60]	Speed: 435.35 samples/sec	Train-multi-accuracy_0=0.968750
INFO:root:Epoch[148] Batch [70]	Speed: 434.35 samples/sec	Train-multi-accuracy_0=0.976562
INFO:root:Epoch[148] Train-multi-accuracy_0=0.978795
INFO:root:Epoch[148] Time cost=22.800


Epoch[148] Validation-multi-accuracy_0=0.508213


INFO:root:Epoch[149] Batch [10]	Speed: 433.26 samples/sec	Train-multi-accuracy_0=0.972301
INFO:root:Epoch[149] Batch [20]	Speed: 434.63 samples/sec	Train-multi-accuracy_0=0.971875
INFO:root:Epoch[149] Batch [30]	Speed: 434.63 samples/sec	Train-multi-accuracy_0=0.981250
INFO:root:Epoch[149] Batch [40]	Speed: 433.60 samples/sec	Train-multi-accuracy_0=0.974219
INFO:root:Epoch[149] Batch [50]	Speed: 433.38 samples/sec	Train-multi-accuracy_0=0.974219
INFO:root:Epoch[149] Batch [60]	Speed: 433.67 samples/sec	Train-multi-accuracy_0=0.971094
INFO:root:Epoch[149] Batch [70]	Speed: 433.48 samples/sec	Train-multi-accuracy_0=0.971094
INFO:root:Epoch[149] Train-multi-accuracy_0=0.985491
INFO:root:Epoch[149] Time cost=22.831


Epoch[149] Validation-multi-accuracy_0=0.512620


In [ ]:
# batch 10 train 0.389, valid: 0.40